In [7]:
import pandas as pd
import sparknlp
from sparknlp.training import CoNLL
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

spark = sparknlp.start()

print('Spark NLP version', sparknlp.version())
print('Apache Spark version', spark.version)


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:489)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1795)
	at org.apache.spark.SparkContext.$anonfun$new$16(SparkContext.scala:533)
	at org.apache.spark.SparkContext.$anonfun$new$16$adapted(SparkContext.scala:533)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:533)
	at org.apache.spark.api.java.JavaSparkC

In [ ]:
train_data = CoNLL().readDataset(spark, './eng.train')
train_data.show(3)

train_data.select(F.explode(F.arrays_zip(train_data.token.result,
                                            train_data.label.result))\
        .alias("cols")) \
        .select(F.expr("cols['0']").alias("token"),
                F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth')\
        .count()\
        .orderBy('count', ascending=False)\
        .show(100,truncate=False)

23/12/27 14:16:00 WARN TaskSetManager: Stage 0 contains a task of very large size (9058 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[{document, 0, 47...|[{document, 0, 47...|[{token, 0, 1, EU...|[{pos, 0, 1, NNP,...|[{named_entity, 0...|
|     Peter Blackburn|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 4, Pe...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
| BRUSSELS 1996-08-22|[{document, 0, 18...|[{document, 0, 18...|[{token, 0, 7, BR...|[{pos, 0, 7, NNP,...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

23/12/27 14:16:01 WARN TaskSetManager: Stage 1 contains a task of very large si

+------------+------+
|ground_truth|count |
+------------+------+
|O           |169578|
|B-LOC       |7140  |
|B-PER       |6600  |
|B-ORG       |6321  |
|I-PER       |4528  |
|I-ORG       |3704  |
|B-MISC      |3438  |
|I-LOC       |1157  |
|I-MISC      |1155  |
+------------+------+



In [ ]:

graph_folder = "./ner_graphs"
graph_builder = TFNerDLGraphBuilder()\
              .setInputCols(["sentence", "token", "embeddings"]) \
              .setLabelColumn("label")\
              .setGraphFile("auto")\
              .setGraphFolder(graph_folder)\
              .setHiddenUnitsNumber(20)

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

embeddings = BertEmbeddings.pretrained().\
    setInputCols(["sentence", 'token']).\
    setOutputCol("embeddings")

small_bert_L2_768 download started this may take some time.
Approximate size to download 135.3 MB
[OK!]


In [ ]:


nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "bert"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(5)\
    .setRandomSeed(0)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True)\
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    # .setGraphFolder(graph_folder)

ner_pipeline = Pipeline(stages=[embeddings, graph_builder, nerTagger])
ner_model = ner_pipeline.fit(train_data)

23/12/27 15:10:32 WARN TaskSetManager: Stage 18 contains a task of very large size (9058 KiB). The maximum recommended task size is 1000 KiB.


Ner DL Graph Builder configuration:
Graph folder: ./ner_graphs
Graph file name: auto
Build params: {'ntags': 9, 'embeddings_dim': 768, 'nchars': 59, 'lstm_size': 20}
Can't build the tensorflow graph with TF 2 graph factory, attempting TF 1.15 factory
<module 'tensorflow._api.v2.version' from '/home/oem/.local/lib/python3.10/site-packages/tensorflow/_api/v2/version/__init__.py'>


Exception: The tensorflow graphs can't be build.

In [ ]:
test_data = CoNLL().readDataset(spark, './eng.testa').limit(1000)
predictions = ner_model.transform(test_data)

In [ ]:
products = pd.read_csv('./products.csv').to_numpy()

test_products = []
for product in products:
    test_products.append(product[0])

test_data = pd.DataFrame({'text': test_products})
test_data.iteritems = test_data.items
test_data = spark.createDataFrame(test_data)

predicts = ner_model.transform(test_data)

In [ ]:
ner_vis = NerVisualizer()

ner_vis.display(predicts, label_col='entities', document_col='text', labels=['PROD', 'MISC'], output_file='ner_vis.html')

ner_vis.set_label_colors({'PROD':'#008080', 'MISC':'#AFEEEE'})
